<a href="https://colab.research.google.com/github/DJNahata/Style-GANS-Practice/blob/master/Discriminator_Network_Check_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!git clone https://github.com/NVlabs/stylegan.git

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [0]:
cd /content/stylegan/

/content/stylegan


In [0]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
import dnnlib
import dnnlib.tflib as tflib

# NOTE: Do not import any application-specific modules here!
# Specify all network parameters as kwargs.

#----------------------------------------------------------------------------
# Primitive ops for manipulating 4D activation tensors.
# The gradients of these are not necessary efficient or even meaningful.

def _blur2d(x, f=[1,2,1], normalize=True, flip=False, stride=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(stride, int) and stride >= 1

    # Finalize filter kernel.
    f = np.array(f, dtype=np.float32)
    if f.ndim == 1:
        f = f[:, np.newaxis] * f[np.newaxis, :]
    assert f.ndim == 2
    if normalize:
        f /= np.sum(f)
    if flip:
        f = f[::-1, ::-1]
    f = f[:, :, np.newaxis, np.newaxis]
    f = np.tile(f, [1, 1, int(x.shape[1]), 1])

    # No-op => early exit.
    if f.shape == (1, 1) and f[0,0] == 1:
        return x

    # Convolve using depthwise_conv2d.
    orig_dtype = x.dtype
    x = tf.cast(x, tf.float32)  # tf.nn.depthwise_conv2d() doesn't support fp16
    f = tf.constant(f, dtype=x.dtype, name='filter')
    strides = [1, 1, stride, stride]
    x = tf.nn.depthwise_conv2d(x, f, strides=strides, padding='SAME', data_format='NCHW')
    x = tf.cast(x, orig_dtype)
    return x

def _upscale2d(x, factor=2, gain=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >= 1

    # Apply gain.
    if gain != 1:
        x *= gain

    # No-op => early exit.
    if factor == 1:
        return x

    # Upscale using tf.tile().
    s = x.shape
    x = tf.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
    x = tf.tile(x, [1, 1, 1, factor, 1, factor])
    x = tf.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
    return x

def _downscale2d(x, factor=2, gain=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >= 1

    # 2x2, float32 => downscale using _blur2d().
    if factor == 2 and x.dtype == tf.float32:
        f = [np.sqrt(gain) / factor] * factor
        return _blur2d(x, f=f, normalize=False, stride=factor)

    # Apply gain.
    if gain != 1:
        x *= gain

    # No-op => early exit.
    if factor == 1:
        return x

    # Large factor => downscale using tf.nn.avg_pool().
    # NOTE: Requires tf_config['graph_options.place_pruned_graph']=True to work.
    ksize = [1, 1, factor, factor]
    return tf.nn.avg_pool(x, ksize=ksize, strides=ksize, padding='VALID', data_format='NCHW')

#----------------------------------------------------------------------------
# High-level ops for manipulating 4D activation tensors.
# The gradients of these are meant to be as efficient as possible.

def blur2d(x, f=[1,2,1], normalize=True):
    with tf.variable_scope('Blur2D'):
        @tf.custom_gradient
        def func(x):
            y = _blur2d(x, f, normalize)
            @tf.custom_gradient
            def grad(dy):
                dx = _blur2d(dy, f, normalize, flip=True)
                return dx, lambda ddx: _blur2d(ddx, f, normalize)
            return y, grad
        return func(x)

def upscale2d(x, factor=2):
    with tf.variable_scope('Upscale2D'):
        @tf.custom_gradient
        def func(x):
            y = _upscale2d(x, factor)
            @tf.custom_gradient
            def grad(dy):
                dx = _downscale2d(dy, factor, gain=factor**2)
                return dx, lambda ddx: _upscale2d(ddx, factor)
            return y, grad
        return func(x)

def downscale2d(x, factor=2):
    with tf.variable_scope('Downscale2D'):
        @tf.custom_gradient
        def func(x):
            y = _downscale2d(x, factor)
            @tf.custom_gradient
            def grad(dy):
                dx = _upscale2d(dy, factor, gain=1/factor**2)
                return dx, lambda ddx: _downscale2d(ddx, factor)
            return y, grad
        return func(x)

TensorFlow 1.x selected.





In [0]:
def get_weight(shape, gain=np.sqrt(2), use_wscale=False, lrmul=1):
    fan_in = np.prod(shape[:-1]) # [kernel, kernel, fmaps_in, fmaps_out] or [in, out]
    he_std = gain / np.sqrt(fan_in) # He init

    # Equalized learning rate and custom learning rate multiplier.
    if use_wscale:
        init_std = 1.0 / lrmul
        runtime_coef = he_std * lrmul
    else:
        init_std = he_std / lrmul
        runtime_coef = lrmul

    # Create variable.
    init = tf.initializers.random_normal(0, init_std)
    return tf.get_variable('weight', shape=shape, initializer=init) * runtime_coef

In [0]:
#----------------------------------------------------------------------------
# Fully-connected layer.

def dense(x, fmaps, res, **kwargs):
  with tf.variable_scope('Dense%dx%d' %(2**res, 2**res), reuse = tf.AUTO_REUSE):
    if len(x.shape) > 2:
        x = tf.reshape(x, [-1, np.prod([d.value for d in x.shape[1:]])])
    w = get_weight([x.shape[1].value, fmaps], **kwargs)
    w = tf.cast(w, x.dtype)
    return tf.matmul(x, w)

In [0]:
#----------------------------------------------------------------------------
# Convolutional layer.

def conv2d(x, fmaps, kernel, res, **kwargs):
  with tf.variable_scope('Convolution%dx%d' % (2**res,2**res), reuse = tf.AUTO_REUSE):
    assert kernel >= 1 and kernel % 2 == 1
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.cast(w, x.dtype)
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME', data_format='NCHW')


In [0]:
#----------------------------------------------------------------------------
# Fused convolution + scaling.
# Faster and uses less memory than performing the operations separately.

def upscale2d_conv2d(x, fmaps, kernel, res, fused_scale='auto', **kwargs):
  with tf.variable_scope('upscale2d_conv2d%dx%d' % (2**res,2**res), reuse = tf.AUTO_REUSE):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) * 2 >= 128

    # Not fused => call the individual ops directly.
    if not fused_scale:
        return conv2d(upscale2d(x), fmaps, kernel, res, **kwargs)

    # Fused => perform both ops simultaneously using tf.nn.conv2d_transpose().
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.transpose(w, [0, 1, 3, 2]) # [kernel, kernel, fmaps_out, fmaps_in]
    w = tf.pad(w, [[1,1], [1,1], [0,0], [0,0]], mode='CONSTANT')
    w = tf.add_n([w[1:, 1:], w[:-1, 1:], w[1:, :-1], w[:-1, :-1]])
    w = tf.cast(w, x.dtype)
    os = [tf.shape(x)[0], fmaps, x.shape[2] * 2, x.shape[3] * 2]
    return tf.nn.conv2d_transpose(x, w, os, strides=[1,1,2,2], padding='SAME', data_format='NCHW')

In [0]:
def conv2d_downscale2d(x, fmaps, kernel, res, fused_scale='auto', **kwargs):
  with tf.variable_scope('conv2d_downscale2d%dx%d' % (2**res,2**res), reuse = tf.AUTO_REUSE):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) >= 128

    # Not fused => call the individual ops directly.
    if not fused_scale:
        return downscale2d(conv2d(x, fmaps, kernel, res, **kwargs))

    # Fused => perform both ops simultaneously using tf.nn.conv2d().
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.pad(w, [[1,1], [1,1], [0,0], [0,0]], mode='CONSTANT')
    w = tf.add_n([w[1:, 1:], w[:-1, 1:], w[1:, :-1], w[:-1, :-1]]) * 0.25
    w = tf.cast(w, x.dtype)
    return tf.nn.conv2d(x, w, strides=[1,1,2,2], padding='SAME', data_format='NCHW')

In [0]:
#----------------------------------------------------------------------------
# Apply bias to the given activation tensor.

def apply_bias(x, res, lrmul=1):
  with tf.variable_scope('Bias%dx%d' % (2**res,2**res), reuse = tf.AUTO_REUSE):
    b = tf.get_variable('bias', shape=[x.shape[1]], initializer=tf.initializers.zeros()) * lrmul
    b = tf.cast(b, x.dtype)
    if len(x.shape) == 2:
        return x + b
    return x + tf.reshape(b, [1, -1, 1, 1])

In [0]:
#----------------------------------------------------------------------------
# Leaky ReLU activation. More efficient than tf.nn.leaky_relu() and supports FP16.

def leaky_relu(x, alpha=0.2):
    with tf.variable_scope('LeakyReLU'):
        alpha = tf.constant(alpha, dtype=x.dtype, name='alpha')
        @tf.custom_gradient
        def func(x):
            y = tf.maximum(x, x * alpha)
            @tf.custom_gradient
            def grad(dy):
                dx = tf.where(y >= 0, dy, dy * alpha)
                return dx, lambda ddx: tf.where(y >= 0, ddx, ddx * alpha)
            return y, grad
        return func(x)

In [0]:
#----------------------------------------------------------------------------
# Pixelwise feature vector normalization.

def pixel_norm(x, epsilon=1e-8):
    with tf.variable_scope('PixelNorm'):
        epsilon = tf.constant(epsilon, dtype=x.dtype, name='epsilon')
        return x * tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

In [0]:
#----------------------------------------------------------------------------
# Instance normalization.

def instance_norm(x, epsilon=1e-8):
    assert len(x.shape) == 4 # NCHW
    with tf.variable_scope('InstanceNorm'):
        orig_dtype = x.dtype
        x = tf.cast(x, tf.float32)
        x -= tf.reduce_mean(x, axis=[2,3], keepdims=True)
        epsilon = tf.constant(epsilon, dtype=x.dtype, name='epsilon')
        x *= tf.rsqrt(tf.reduce_mean(tf.square(x), axis=[2,3], keepdims=True) + epsilon)
        x = tf.cast(x, orig_dtype)
        return x

In [0]:
#----------------------------------------------------------------------------
# Style modulation.

def style_mod(x, dlatent, res, **kwargs):
    with tf.variable_scope('StyleMod', reuse=tf.AUTO_REUSE):
        style = apply_bias(dense(dlatent, fmaps=x.shape[1]*2, res=res, gain=1, **kwargs), res)
        style = tf.reshape(style, [-1, 2, x.shape[1]] + [1] * (len(x.shape) - 2))
        return x * (style[:,0] + 1) + style[:,1]

In [0]:
#----------------------------------------------------------------------------
# Noise input.

def apply_noise(x, res, noise_var=None, randomize_noise=True):
    assert len(x.shape) == 4 # NCHW
    with tf.variable_scope('Noise%dx%d' %(2**res,2**res), reuse=tf.AUTO_REUSE):
        if noise_var is None or randomize_noise:
            noise = tf.random_normal([tf.shape(x)[0], 1, x.shape[2], x.shape[3]], dtype=x.dtype)
        else:
            noise = tf.cast(noise_var, x.dtype)
        weight = tf.get_variable('weight', shape=[x.shape[1].value], initializer=tf.initializers.zeros())
        return x + noise * tf.reshape(tf.cast(weight, x.dtype), [1, -1, 1, 1])

In [0]:
def minibatch_stddev_layer(x, group_size=4, num_new_features=1):
    with tf.variable_scope('MinibatchStddev'):
        group_size = tf.minimum(group_size, tf.shape(x)[0])     # Minibatch must be divisible by (or smaller than) group_size.
        s = x.shape                                             # [NCHW]  Input shape.
        y = tf.reshape(x, [group_size, -1, num_new_features, s[1]//num_new_features, s[2], s[3]])   # [GMncHW] Split minibatch into M groups of size G. Split channels into n channel groups c.
        y = tf.cast(y, tf.float32)                              # [GMncHW] Cast to FP32.
        y -= tf.reduce_mean(y, axis=0, keepdims=True)           # [GMncHW] Subtract mean over group.
        y = tf.reduce_mean(tf.square(y), axis=0)                # [MncHW]  Calc variance over group.
        y = tf.sqrt(y + 1e-8)                                   # [MncHW]  Calc stddev over group.
        y = tf.reduce_mean(y, axis=[2,3,4], keepdims=True)      # [Mn111]  Take average over fmaps and pixels.
        y = tf.reduce_mean(y, axis=[2])                         # [Mn11] Split channels into c channel groups
        y = tf.cast(y, x.dtype)                                 # [Mn11]  Cast back to original data type.
        y = tf.tile(y, [group_size, 1, s[2], s[3]])             # [NnHW]  Replicate over group and pixels.
        return tf.concat([x, y], axis=1)

In [0]:
def D_basic(
    images_in,                          # First input: Images [minibatch, channel, height, width].
    labels_in,                          # Second input: Labels [minibatch, label_size].
    num_channels        = 3,            # Number of input color channels. Overridden based on dataset.
    resolution          = 256,           # Input resolution. Overridden based on dataset.
    label_size          = 0,            # Dimensionality of the labels, 0 if no labels. Overridden based on dataset.
    fmap_base           = 8192,         # Overall multiplier for the number of feature maps.
    fmap_decay          = 1.0,          # log2 feature map reduction when doubling the resolution.
    fmap_max            = 512,          # Maximum number of feature maps in any layer.
    nonlinearity        = 'lrelu',      # Activation function: 'relu', 'lrelu',
    use_wscale          = True,         # Enable equalized learning rate?
    mbstd_group_size    = 4,            # Group size for the minibatch standard deviation layer, 0 = disable.
    mbstd_num_features  = 1,            # Number of features for the minibatch standard deviation layer.
    dtype               = 'float32',    # Data type to use for activations and outputs.
    fused_scale         = 'auto',       # True = fused convolution + scaling, False = separate ops, 'auto' = decide automatically.
    blur_filter         = [1,2,1],      # Low-pass filter to apply when resampling activations. None = no filtering.
    structure           = 'auto',       # 'fixed' = no progressive growing, 'linear' = human-readable, 'recursive' = efficient, 'auto' = select automatically.
    is_template_graph   = False,        # True = template graph constructed by the Network class, False = actual evaluation.
    **_kwargs):                         # Ignore unrecognized keyword args.

    resolution_log2 = int(np.log2(resolution))
    assert resolution == 2**resolution_log2 and resolution >= 4
    def nf(stage): return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
    def blur(x): return blur2d(x, blur_filter) if blur_filter else x
    if structure == 'auto': structure = 'linear' if is_template_graph else 'recursive'
    act, gain = {'relu': (tf.nn.relu, np.sqrt(2)), 'lrelu': (leaky_relu, np.sqrt(2))}[nonlinearity]

    images_in.set_shape([None, num_channels, resolution, resolution])

    if labels_in is not None:
        labels_in.set_shape([None, label_size])
        labels_in = tf.cast(labels_in, dtype)

    images_in = tf.cast(images_in, dtype)
    with tf.variable_scope('lod', reuse = tf.AUTO_REUSE):
        lod_in = tf.cast(tf.get_variable('lod', initializer=np.float32(0.0), trainable=False), dtype)


    scores_out = None

    def fromrgb(x, res): # res = 2..resolution_log2
        with tf.variable_scope('FromRGB_lod%d' % (resolution_log2 - res)):
            return act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=1, res = res, gain=gain, use_wscale=use_wscale), res))

    def block(x, res): # res = 2..resolution_log2
        if res >= 3: # 8x8 and up
            with tf.variable_scope('Conv0'):
                x = act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=3, res = res, gain=gain, use_wscale=use_wscale), res))
            with tf.variable_scope('Conv1_down'):
                x = act(apply_bias(conv2d_downscale2d(blur(x), fmaps=nf(res-2), kernel=3, res = res, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale), res))
        else: # 4x4
            if mbstd_group_size > 1:
                x = minibatch_stddev_layer(x, mbstd_group_size, mbstd_num_features)
            with tf.variable_scope('Convol'):
                x = act(apply_bias(conv2d(x, fmaps=nf(res-1), kernel=3, res = 2, gain=gain, use_wscale=use_wscale), res))
            with tf.variable_scope('Dense0'):
                x = act(apply_bias(dense(x, fmaps=nf(res-2), res = 2, gain=gain, use_wscale=use_wscale), 2))
            with tf.variable_scope('Dense1'):
                x = apply_bias(dense(x, fmaps=max(label_size, 1), res = 2, gain=1, use_wscale=use_wscale), 2)
        return x


    if structure == 'fixed':
        x = fromrgb(images_in, resolution_log2)
        for res in range(resolution_log2, 2, -1):
            x = block(x, res)
        scores_out = block(x, 2)


    # Linear structure: simple but inefficient.
    if structure == 'linear':
        img = images_in
        x = fromrgb(img, resolution_log2)
        for res in range(resolution_log2, 2, -1):
            lod = resolution_log2 - res
            x = block(x, res)
            img = downscale2d(img)
            y = fromrgb(img, res - 1)
            with tf.variable_scope('Grow_lod%d' % lod):
                x = tflib.lerp_clip(x, y, lod_in - lod)
        scores_out = block(x, 2)


    # Recursive structure: complex but efficient.
    if structure == 'recursive':
        def cset(cur_lambda, new_cond, new_lambda):
            return lambda: tf.cond(new_cond, new_lambda, cur_lambda)
        def grow(res, lod):
            x = lambda: fromrgb(downscale2d(images_in, 2**lod), res)
            if lod > 0: x = cset(x, (lod_in < lod), lambda: grow(res + 1, lod - 1))
            x = block(x(), res); y = lambda: x
            if res > 2: y = cset(y, (lod_in > lod), lambda: tflib.lerp(x, fromrgb(downscale2d(images_in, 2**(lod+1)), res - 1), lod_in - lod))
            return y()
        scores_out = grow(2, resolution_log2 - 2)
    
    print(scores_out.shape)

    # Label conditioning from "Which Training Methods for GANs do actually Converge?"
    if label_size:
        with tf.variable_scope('LabelSwitch'):
            scores_out = tf.reduce_sum(scores_out * labels_in, axis=1, keepdims=True)

    assert scores_out.dtype == tf.as_dtype(dtype)
    scores_out = tf.identity(scores_out, name='scores_out')
    return scores_out

In [0]:
a1 = np.random.random((1,3,256,256))
a1 = tf.convert_to_tensor(a1, None)
a2 = D_basic(a1, None)
print(a2.shape)

(1, 1)
(1, 1)


In [0]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [0]:
print(sess.run(a2))

[[0.04991315]]


In [0]:
a211 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Conv0/Conv')
a212 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Conv0/Bias')
a221 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Conv1_down/conv2d_downscale2d')
a222 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Conv1_down/Bias')
a231 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Convol/Convolution')
a232 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Convol/Bias')
a241 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Dense0/Dense')
a242 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Dense0/Bias')
a251 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Dense1/Dense')
a252 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Dense1/Bias')

In [0]:
print(len(a211))
print(len(a212))
print(len(a221))
print(len(a222))
print(len(a231))
print(len(a232))
print(len(a241))
print(len(a242))
print(len(a251))
print(len(a252))

6
6
6
6
1
1
1
1
1
1


In [0]:
for i in a211:
    print(i)
print(" ")
for i in a212:
    print(i)
print()
for i in a221:
    print(i)
print()
for i in a222:
    print(i)
print()
print(a231[0])
print()
print(a232[0])
print()
print(a241[0])
print()
print(a242[0])
print()
print(a251[0])
print()
print(a252[0])
print()

<tf.Variable 'Conv0/Convolution256x256/weight:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'Conv0/Convolution128x128/weight:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'Conv0/Convolution64x64/weight:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'Conv0/Convolution32x32/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>
<tf.Variable 'Conv0/Convolution16x16/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>
<tf.Variable 'Conv0/Convolution8x8/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>
 
<tf.Variable 'Conv0/Bias256x256/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'Conv0/Bias128x128/bias:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'Conv0/Bias64x64/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'Conv0/Bias32x32/bias:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'Conv0/Bias16x16/bias:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'Conv0/Bias8x8/bias:0' shape=(512,) dtype=float32_ref>

<tf.Variable 'Conv1_down/conv2d_downscale2

In [0]:
print(a221)

[<tf.Variable 'Conv1_down/conv2d_downscale2d256x256/weight:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'Conv1_down/conv2d_downscale2d128x128/weight:0' shape=(3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'Conv1_down/conv2d_downscale2d64x64/Convolution64x64/weight:0' shape=(3, 3, 256, 512) dtype=float32_ref>, <tf.Variable 'Conv1_down/conv2d_downscale2d32x32/Convolution32x32/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>, <tf.Variable 'Conv1_down/conv2d_downscale2d16x16/Convolution16x16/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>, <tf.Variable 'Conv1_down/conv2d_downscale2d8x8/Convolution8x8/weight:0' shape=(3, 3, 512, 512) dtype=float32_ref>]
